In [ ]:

    "# Dynamic Arrays: A Simple Data Structure\n",
    "These notes will supplement section 17.4 of CLRS book, which covers a lot more than this chapter.\n",
    
    
    "A dynamic array is simply an array but it can grow in size to accomodate new elements that are added. \n",
    "\n",
    "## Arrays \n",
    "\n",
    "An array is a \"contiguous chunk\" of random access memory in a computer.  \n",
    "  - Random Access: We can access the individual cells of the array as `a[1]`, `a[2]`, ..., `a[n]`, where  $n$ is the size of the array. \n",
    "  - Reading or writing to the memory element at index `j`  takes $O(1)$ time. \n",
    "  \n",
    "Our goal is to maintain an array of $n$ elements and support the following operations:\n",
    "  - Reading/Writing to a particular index $j$ where $1 \\leq j \\leq n$.\n",
    "  - Adding a new element at the end of the array: the size of the array will become $n+1$ as a result.\n",
    "  \n",
    "  \n",
    "## Memory Allocator\n",
    "\n",
    "The main difficulty in implementing an array data structure lies in how a process in a computer obtains memory. In all computer operating systems there is a memory management module that allocates memory to running programs. Programs can request a \"contiguous chunk\" of $k$ memory cells using an \"allocation\" function. This function is setup differently in various programming languages. For instance, in python, we can allocate an array of size `k` all initialized with $0$s as follows:\n",
    "\n",
    "~~~\n",
    "a = [0]* k\n",
    "~~~\n",
    "\n",
    "Note however, that lists in python are already a \"dynamic array\" implemented more or less in the same manner that we are going to describe here. \n",
    "\n",
    "https://stackoverflow.com/questions/3917574/how-is-pythons-list-implemented\n",
    "\n",
    "The curious reader may ask about \"deallocation\" or \"freeing\" memory. We note that \n",
    "in some programming environments like C/C++ this is required for the program to explicitly tell the operating system that a particular chunk of memory that was previously allocated is no longer needed. However, python is a _garbage collected_ language. I.e, the python runtime manages memory and decides that a chunk of memory is no longer needed/can be freed. The details of garbage collection are beyond the scope of this course. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "04841dc6",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Allocate a new memory of size `size`\n",
    "def allocateMemory(size): \n",
    "    assert size >= 1\n",
    "    return [0]*size\n",
    "\n",
    "# Copy the contents of old list into new\n",
    "def copyInto(old, new):\n",
    "    assert len(old) <= len(new), 'Not enough space to copy into'\n",
    "    m = len(old)\n",
    "    for i in range(m):\n",
    "        new[i] = old[i]"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "2301989c",
   "metadata": {},
   "source": [
    "We will now implement the `DynamicArray` data structure as a Python class. \n",
    "It will have three fields:\n",
    "  - `array`: the overall memory that has been allocated.\n",
    "  - `allocated_size`: How much is the allocated size?\n",
    "  - `size`: what is the actual size of the array?\n",
    "  \n",
    "Note that `allocated_size` is always larger than the actual size. For instance, \n",
    "`allocated_size=32` means that `32` cells have been allocated. However, `size=10` means that only 10 out of the 32 cells are used by the array.\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "id": "b1ce30ae",
   "metadata": {},
   "outputs": [],
   "source": [
    "class DynamicArray: \n",
    "    \n",
    "    def __init__(self, initial_size=16, initial_fill=0, debug=False):\n",
    "        self.allocated_size = initial_size \n",
    "        self.size = 0\n",
    "        self.array = [initial_fill] * initial_size\n",
    "        self.debug = debug\n",
    "    \n",
    "    # This allows us to directly access d[idx]\n",
    "    def __getitem__(self, idx):\n",
    "        assert idx >= 0 and idx < self.size \n",
    "        return self.array[idx]\n",
    "    \n",
    "    # This allows us to write d[idx] = val \n",
    "    def __setitem__(self, idx, val):\n",
    "        assert idx >= 0 and idx < self.size \n",
    "        self.array[idx] = val\n",
    "    \n",
    "    def append(self, x):\n",
    "        # Do we have enough allocated size to just append x to the array?\n",
    "        if self.size >= self.allocated_size:\n",
    "            if self.debug: \n",
    "                print(f'Ran out of memory: old allocated size: {self.allocated_size}, new allocated size is {2*self.allocated_size}')\n",
    "            # No, we have run out of pre-allocated memory\n",
    "            # Double the size of the array \n",
    "            # Double the size of the allocated memory\n",
    "            self.allocated_size = 2 * self.allocated_size\n",
    "            old_array = self.array\n",
    "            # allocate and copy.\n",
    "            new_array = allocateMemory(self.allocated_size)\n",
    "            copyInto(old_array, new_array)\n",
    "            # update the array.\n",
    "            self.array = new_array\n",
    "        # Append the element to the end\n",
    "        self.array[self.size] = x\n",
    "        # Update its size.\n",
    "        self.size = self.size + 1\n",
    "    "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "id": "1f62a4a4",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Ran out of memory: old allocated size: 1, new allocated size is 2\n",
      "Ran out of memory: old allocated size: 2, new allocated size is 4\n",
      "Ran out of memory: old allocated size: 4, new allocated size is 8\n",
      "Ran out of memory: old allocated size: 8, new allocated size is 16\n",
      "Ran out of memory: old allocated size: 16, new allocated size is 32\n",
      "Ran out of memory: old allocated size: 32, new allocated size is 64\n",
      "Ran out of memory: old allocated size: 64, new allocated size is 128\n",
      "Ran out of memory: old allocated size: 128, new allocated size is 256\n",
      "Ran out of memory: old allocated size: 256, new allocated size is 512\n",
      "Ran out of memory: old allocated size: 512, new allocated size is 1024\n",
      "l[5] = 5\n",
      "l[0] = 30\n"
     ]
    }
   ],
   "source": [
    "l = DynamicArray(initial_size=1, initial_fill=0, debug=True)\n",
    "for j in range(1000):\n",
    "    l.append(j)\n",
    "print(f'l[5] = {l[5]}')\n",
    "l[0] = 30\n",
    "print(f'l[0] = {l[0]}')\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0fdff351",
   "metadata": {},
   "source": [
    "Suppose we have appended $n$ elements so far to the array. How many times have we had to grow, assuming the initial allocated size is $1$?\n",
    "\n",
    "- Allocated size grows from 1 to 2\n",
    "- Allocated size grows from 2 to 4\n",
    "- ..\n",
    "- Allocated size grows from $2^{k}$ to $2^{k+1}$ where $n< 2^{k+1}$.\n",
    "\n",
    "In other words, we grow $k+1$ times, where $2^{k} \\leq n < 2^{k+1}$.\n",
    "We conclude that $k = \\lceil \\log_2(n) \\rceil$.\n",
    "\n",
    "However, when we reallocate from size $m$ to $2m$, we have to copy over $m$ elements of the array. Therefore, the total work required to append $n$ elements is given by :\n",
    "\n",
    "- Allocated size grows from 1 to 2 -- `1 unit of time`.\n",
    "- Allocated size grows from 2 to 4 -- `2 units of time`\n",
    "- ..\n",
    "- Allocated size grows from $2^{k}$ to $2^{k+1}$ where $n< 2^{k+1}$ - $2^k$ ` units of time`.\n",
    "\n",
    "Total time needed for all the reallocations: $1 + 2 + \\cdots 2^k = 2^{k+1} -1  \\leq  2 n -1 $.\n",
    "\n",
    "Also, each append requires $1$ unit of time to copy the element and update the size.\n",
    "\n",
    "Thus, adding all of it up: appending $n$ elements from scratch requires $3n$ units of time.\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}